In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from operator import itemgetter

In [7]:
# 체인에 메모리를 추가하는 방법을 보여줍니다. 현재 메모리 클래스를 사용할 수 있지만 수동으로 연결해야 합니다

In [4]:
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [6]:
# 대화형 프롬프트를 생성합니다. 
# 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [8]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
# 키로 chat_history를 맵핑
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

C:\Users\lee\AppData\Local\Temp\ipykernel_12536\3425915295.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [9]:
memory.load_memory_variables({})  # chat_history가 키가 된다.

{'chat_history': []}

In [10]:
# chat_history라는 키로 RunnableLambda(memory.load_memory_variables)를 맵핑.
# RunnableLambda는 memory.load_memory_variables({})라는 이 함수를 실행한다. 
# 지금까지 대화기롤을 불러오는 함수를 호출하게 된다. 'chat_history': {'chat_history': []} 
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) 
)

In [11]:
print(runnable.invoke({"input": "hi"}))

{'input': 'hi', 'chat_history': {'chat_history': []}}


In [12]:
# chat_history라는 키로 RunnableLambda(memory.load_memory_variables)를 맵핑.
# RunnableLambda는 memory.load_memory_variables({})이 함수를 실행한다. 
# 지금까지 대화기롤을 불러오는 함수를 호출하게 된다. {'chat_history': []} 
# 'chat_history' : {'chat_history': []} 
# itemgetter("chat_history")는 'chat_history': {'chat_history': []} 딕셔너리에서 
# 키를 chat_history로 넣어서 [] 리스트(대화기록)을 가져온다.
runnable = RunnablePassthrough.assign(
    # memory_key(chat_history)와 동일 하게 입력한다.
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")    	
)

In [13]:
print(runnable.invoke({"input": "hi"}))

{'input': 'hi', 'chat_history': []}


In [14]:
# runnable의 결과 {'input': 'hi', 'chat_history': []} 이 프롬프트로 입력
# input에는 hi, chat_history에는 []가 들어간다. 자동으로 채워진다.
# 그리고 모델로 전달된다.
chain = runnable | prompt | model

In [15]:
# chain 객체의 invoke 메서드를 사용하여 입력에 대한 응답을 생성합니다.
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 이인환입니다."})
print(response.content)

안녕하세요, 이인환님! 만나서 반갑습니다. 어떻게 도와드릴까요?


In [16]:
print(memory.load_memory_variables({}))    

{'chat_history': []}


In [17]:
# 입력된 데이터와 응답 내용을 메모리에 저장합니다.
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 이인환입니다."}, 
	{"ai": response.content}
)

In [18]:
# 저장된 대화기록을 출력합니다.
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 이인환입니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요, 이인환님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={})]}

In [20]:
# 이름을 기억하고 있는지 추가 질의합니다.
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
print(response.content)

네, 이인환님이라고 하셨습니다. 다른 질문이나 도움이 필요하신 부분이 있으면 말씀해 주세요!
